In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import pandas as pd
import os
import gzip
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt

In [9]:
# tf.function：   python编辑的代码块 -> TensorFlow的图  
# autograph：     tf.function所依赖的机制

# python函数
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0  -> scale * z
    # z < 0   -> scale * alpha * tf.nn.elu(z)
    is_positive  = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5, 1])))

# tf.function转为TensorFlow中图实现的函数
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5, 1])))

# 也可以转回
print(scaled_elu_tf.python_function is scaled_elu)

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915    1.        ], shape=(3,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915    1.        ], shape=(3,), dtype=float32)
True


In [10]:
# 速度优势
%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

8.23 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
6.92 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
# 另一种放法转换
# 1 + 1/2 + 1/2^2 + ... + 1/2^n
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf. constant(1.)
    for i in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [32]:
# 与视频不一致
def display_tf_code(func):
    # 获得源代码
    code = tf.autograph.to_code(func)
    # 用Markdown展示源码
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [15]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_positive = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_positive, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [16]:
display_tf_code(converge_to_2)

ConversionError: converting <tensorflow.python.eager.def_function.Function object at 0x000001E808AC1E10>: AttributeError: 'Function' object has no attribute '__code__'

In [19]:
# tf.variable只能在函数外面先初始化
# 否则ValueError: tf.function-decorated function tried to create variables on non-first call.
var = tf.Variable(0.)
@tf.function
def add_21():
    return var.assign_add(21)  # +=
print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


函数签名与图结构
用处：
1）如何保存模型
2）如何载入已经保存好的模型，做inference

In [21]:
#加入类型限制
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)
    
print(cube(tf.constant([1, 2, 3])))

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))
tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)


In [33]:
# get_concrete_function  -> add input signature -> SavedModel
# eager.function.ConcreteFunction对象
cube_func_int32 = cube.get_concrete_function(      # 需要传入输入类型
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)
print(cube_func_int32 is cube.get_concrete_function(tf.TensorSpec([5], tf.int32)))
print(cube_func_int32 is cube.get_concrete_function(tf.constant([1, 2, 3])))

True
True


In [24]:
# 对eager.function.ConcreteFunction对象的操作
# 1）对图的操作
cube_func_int32.graph

In [26]:
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [27]:
pow_op = cube_func_int32.graph.get_operations()[2]
print(pow_op)

name: "Pow"
op: "Pow"
input: "x"
input: "Pow/y"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}



In [28]:
print(list(pow_op.inputs))
print(list(pow_op.outputs))
# get_operations()前两个操作的输出是第三个操作的输入
#                 第三个输出是第四个的输入

[<tf.Tensor 'x:0' shape=(None,) dtype=int32>, <tf.Tensor 'Pow/y:0' shape=() dtype=int32>]
[<tf.Tensor 'Pow:0' shape=(None,) dtype=int32>]


In [29]:
cube_func_int32.graph.get_operation_by_name("x")
# Placeholder：用来放置输入
# 1.0需要现实定义，2.0就不用 但依旧在图模型中存在

<tf.Operation 'x' type=Placeholder>

In [30]:
cube_func_int32.graph.get_tensor_by_name("x:0")

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [31]:
# 粗暴的直到层的名字
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}